In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv("agr_en_train.csv",header = None )
data.columns = ['id', 'text', 'label']

# Look at the first few rows of the dataframe
display (data.head())

print (data.info())




,id,text,label
0,facebook_corpus_msr_1723796,Well said sonu..you have courage to stand agai...,OAG
1,facebook_corpus_msr_466073,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG
2,facebook_corpus_msr_1493901,"Now question is, Pakistan will adhere to this?",OAG
3,facebook_corpus_msr_405512,Pakistan is comprised of fake muslims who does...,OAG
4,facebook_corpus_msr_1521685,"??we r against cow slaughter,so of course it w...",NAG


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11999 non-null  object
 1   text    11999 non-null  object
 2   label   11999 non-null  object
dtypes: object(3)
memory usage: 281.4+ KB
None


In [2]:
# check my dataset balanced or not \
print(data['label'].value_counts())
# not very inbalanced ??

NAG    5051
CAG    4240
OAG    2708
Name: label, dtype: int64


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.7, min_df=5, stop_words='english')

# Fit and transform the training data 
tfidf_train = tfidf_vectorizer.fit_transform(data['text'])


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_train, data['label'], test_size=0.2, random_state=42)


display (X_train)
display (y_train)
display (tfidf_train)

<9599x4630 sparse matrix of type '<class 'numpy.float64'>'
	with 92302 stored elements in Compressed Sparse Row format>

9182     CAG
10487    NAG
6428     NAG
288      CAG
2626     CAG
        ... 
11964    NAG
5191     NAG
5390     CAG
860      CAG
7270     NAG
Name: label, Length: 9599, dtype: object

<11999x4630 sparse matrix of type '<class 'numpy.float64'>'
	with 114422 stored elements in Compressed Sparse Row format>

In [6]:
# an dummy classification 
from sklearn.dummy import DummyClassifier 


dummyClassifier = DummyClassifier(strategy="most_frequent")
dummyClassifier.fit(X_train, y_train)
print ("most frequent ",dummyClassifier.score(X_test,y_test))


dummyClassifier = DummyClassifier(strategy="stratified")
dummyClassifier.fit(X_train, y_train)
print ("stratified",dummyClassifier.score(X_test,y_test))


dummyClassifier = DummyClassifier(strategy="uniform")
dummyClassifier.fit(X_train, y_train)
print ("uniform ",dummyClassifier.score(X_test,y_test))

most frequent  0.4275
stratified 0.34958333333333336
uniform  0.32666666666666666


In [11]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = dummyClassifier.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy:  0.31916666666666665
              precision    recall  f1-score   support

         CAG       0.34      0.31      0.33       867
         NAG       0.42      0.32      0.36      1026
         OAG       0.21      0.34      0.26       507

    accuracy                           0.32      2400
   macro avg       0.32      0.32      0.31      2400
weighted avg       0.35      0.32      0.33      2400



In [9]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression classifier
clf = LogisticRegression(multi_class='ovr')  # 'ovr' stands for One-vs-Rest

# Fit the classifier
clf.fit(X_train, y_train)


LogisticRegression(multi_class='ovr')

In [10]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy:  0.5725
              precision    recall  f1-score   support

         CAG       0.51      0.51      0.51       867
         NAG       0.62      0.74      0.67      1026
         OAG       0.55      0.34      0.42       507

    accuracy                           0.57      2400
   macro avg       0.56      0.53      0.54      2400
weighted avg       0.57      0.57      0.56      2400



In [7]:
def predict_new_text(text):
    tfidf_text = tfidf_vectorizer.transform([text])
    return clf.predict(tfidf_text)

# Example usage
new_text = "Your example text here."
print("Prediction: ", predict_new_text(new_text))


Prediction:  ['OAG']


In [8]:
# Function to remove non-ASCII characters
def remove_non_ascii(text):
    return ''.join(i for i in text if ord(i) < 128)




In [9]:
# read data 
def readData (name):
    df = pd.read_json(name, lines = True)


    # timestamp_ms is same with created time , droped as it have littel meaning
    
    # rememeber to undrop created_at

    newDf = df.drop(columns =['id','id_str','in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id','in_reply_to_user_id_str'
                              ,'in_reply_to_screen_name','source','user','geo','coordinates','place','contributors','quote_count','reply_count'
                              ,'is_quote_status','retweet_count','favorite_count','favorited','retweeted','possibly_sensitive','withheld_in_countries'
                              ,'extended_entities','quoted_status_permalink','quoted_status','quoted_status_id_str','quoted_status_id','retweeted_status',
                             'extended_tweet','display_text_range','timestamp_ms','filter_level','entities','truncated'])
    # only need english data at this stage 
    filtered_df = newDf[newDf['lang'] == "en"]
    # Apply the function to the 'text' column
    filtered_df['text'] = filtered_df['text'].apply(remove_non_ascii)

    print ('\ndataset ')
    display(filtered_df)
    return filtered_df


classfierData= readData ('20221117000100.json')


dataset 


/var/folders/wb/k4gvn4054lvctzf9spdy5c5c0000gn/T/ipykernel_94056/2239758755.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['text'] = filtered_df['text'].apply(remove_non_ascii)


,created_at,text,lang
0,2022-11-17 00:00:52+00:00,Check out Felipe Hildebrand's video! #TikTok h...,en
2,2022-11-17 00:00:52+00:00,@ElectionWiz @tboneMASS I Wish I discovered th...,en
3,2022-11-17 00:00:52+00:00,@durjoy Why is nobody talking about this? Unis...,en
4,2022-11-17 00:00:52+00:00,@ggreenwald @fye336 I Wish I discovered this e...,en
6,2022-11-17 00:00:52+00:00,@bariweiss @AmandaLuvsRoses Why is nobody talk...,en
...,...,...,...
2747,2022-11-17 00:01:52+00:00,RT @shiluvankuna: Remove one thing https://t....,en
2748,2022-11-17 00:01:52+00:00,The course of true love never did run sm\n\n h...,en
2750,2022-11-17 00:01:52+00:00,Sidney Ellen Sylvia Brown # Doris # Milo Bruno...,en
2751,2022-11-17 00:01:52+00:00,RT @davemacladd: I dont think its right to cal...,en


In [10]:
# Example usage
classfierData['processed'] = classfierData['text'].apply(predict_new_text)
agg_lan  = classfierData[classfierData['processed'] == 'OAG']

In [13]:
display (agg_lan)

# 945 get 20 
# 1921 looks like not an aggressive language 
# 821 is 

,created_at,text,lang,processed
106,2022-11-17 00:00:54+00:00,RT @DaveyostsHat: @darreldrowland Gary isnt ou...,en,[OAG]
133,2022-11-17 00:00:55+00:00,RT @BDTRELILBROTHER: I be so happy Im not in c...,en,[OAG]
157,2022-11-17 00:00:55+00:00,It doesn't matter what kind of people we want ...,en,[OAG]
586,2022-11-17 00:01:04+00:00,"Companion? Friend lovers, transformations cann...",en,[OAG]
812,2022-11-17 00:01:09+00:00,why did i forget about andor i might not have ...,en,[OAG]
821,2022-11-17 00:01:09+00:00,Do you know if pigs have periods?' Are you kid...,en,[OAG]
1337,2022-11-17 00:01:20+00:00,This really does capture the essence of k-on p...,en,[OAG]
1427,2022-11-17 00:01:22+00:00,@Victorshi2020 Lmfao what? You sound like a idiot,en,[OAG]
1486,2022-11-17 00:01:24+00:00,RT @pradeeponelife: Also few of the posts are ...,en,[OAG]
1703,2022-11-17 00:01:29+00:00,RT @TeoHanss: @ercep10 @sportbible Shut up htt...,en,[OAG]
